In [2]:
import pandas as pd
import numpy as np
import itertools

In [3]:
df = pd.read_csv('/dsa/groups/casestudy2022su/team06/TourneyMasterFile.csv')

In [4]:
df.head()

,Unnamed: 0,tournament_id,player_id,date,golfer,tournament,course,purse,year,score,...,finish,rounds,cut,shots_gained_putt,shots_gained_around_green,shots_gained_approach_green,shots_gained_off_the_tee,shots_gained_tee_to_green,shots_gained_total,top10
0,1,401353254,9261,3/13/2022,Abraham Ancer,The Players Championship,"TPC Sawgrass - Ponte Vedra Beach, FL",20.0,2022,285,...,T33,4,1,0.16,0.04,0.70,0.20,0.94,1.09,0
1,2,401353254,5548,3/13/2022,Adam Hadwin,The Players Championship,"TPC Sawgrass - Ponte Vedra Beach, FL",20.0,2022,281,...,T9,4,1,1.43,-0.11,1.04,-0.27,0.67,2.09,1
2,3,401353254,4989,3/13/2022,Anirban Lahiri,The Players Championship,"TPC Sawgrass - Ponte Vedra Beach, FL",20.0,2022,276,...,2,4,1,1.41,0.02,1.04,0.87,1.93,3.34,1
3,4,401353254,6015,3/13/2022,Adam Long,The Players Championship,"TPC Sawgrass - Ponte Vedra Beach, FL",20.0,2022,287,...,T46,4,1,0.37,0.15,-0.61,0.67,0.22,0.59,0
4,5,401353254,3832,3/13/2022,Alexander Noren,The Players Championship,"TPC Sawgrass - Ponte Vedra Beach, FL",20.0,2022,284,...,T26,4,1,-0.15,0.08,1.25,0.16,1.49,1.34,0


In [4]:
df = df[['tournament_id','player_id','date','golfer','cut','top10']]

In [5]:
df['date'] =  pd.to_datetime(df['date'])

In [6]:
df.dtypes

tournament_id             int64
player_id                 int64
date             datetime64[ns]
golfer                   object
cut                       int64
top10                     int64
dtype: object

In [7]:
df.head()

,tournament_id,player_id,date,golfer,cut,top10
0,401353254,9261,2022-03-13,Abraham Ancer,1,0
1,401353254,5548,2022-03-13,Adam Hadwin,1,1
2,401353254,4989,2022-03-13,Anirban Lahiri,1,1
3,401353254,6015,2022-03-13,Adam Long,1,0
4,401353254,3832,2022-03-13,Alexander Noren,1,0


In [8]:
df_sorted = df.sort_values(by=['golfer','date'],ascending=True)

In [9]:
df_sorted.head()
#will the date in this format be able to be merged into other dates?
#will we need to change our date column to date datatype?

,tournament_id,player_id,date,golfer,cut,top10
35434,2271,16,2014-10-12,Aaron Baddeley,1,0
35320,2260,16,2014-10-19,Aaron Baddeley,1,0
35218,2276,16,2014-10-26,Aaron Baddeley,1,0
34926,2270,16,2014-11-16,Aaron Baddeley,1,0
34538,2232,16,2015-02-01,Aaron Baddeley,1,0


In [10]:
df_sorted = df_sorted.reset_index(drop=True)

In [11]:
df_sorted.head()

,tournament_id,player_id,date,golfer,cut,top10
0,2271,16,2014-10-12,Aaron Baddeley,1,0
1,2260,16,2014-10-19,Aaron Baddeley,1,0
2,2276,16,2014-10-26,Aaron Baddeley,1,0
3,2270,16,2014-11-16,Aaron Baddeley,1,0
4,2232,16,2015-02-01,Aaron Baddeley,1,0


In [12]:
def generate_streak_counter(golf_df,streak_type):
    '''
    This counts the number cuts made/missed by a golfer in a row from tournament to tournament.
    The dataframe being input is the tournament df that has been cleaned from our file share.
    This function will use the cuts column to differentiate the streak and group by player_id
    to make sure the right player's streak is being counted.
    
    This function will return the original df with a new column that counts the streak.
    
    The streak_type input can be either 'cut' or 'top10' depending on the type of streak 
    you are looking to investigate.
    
    '''
    
    data = golf_df[['player_id',streak_type]].copy()
    data['start_of_streak_{}'.format(streak_type)] = data[streak_type].ne(data[streak_type].shift())
    data['streak_id_{}'.format(streak_type)] = data['start_of_streak_{}'.format(streak_type)].cumsum()
    data['streak_counter_{}'.format(streak_type)] = data.groupby(['player_id','streak_id_{}'.format(streak_type)]).cumcount() +1
#     print(data['streak_counter_{}'.format(streak_type)])
    
#     shots_with_streaks = pd.concat([df_sorted, data['streak_counter_{}'.format(streak_type)]], axis =1)
#     return shots_with_streaks
    return data['streak_counter_{}'.format(streak_type)]

golf_df = df_sorted
golfer_streaks = generate_streak_counter(golf_df,'cut')



In [13]:
golfer_streaks.head(30)

0     1
1     2
2     3
3     4
4     5
5     1
6     1
7     1
8     1
9     1
10    1
11    1
12    1
13    1
14    2
15    1
16    1
17    2
18    3
19    1
20    1
21    1
22    1
23    1
24    2
25    1
26    1
27    2
28    3
29    1
Name: streak_counter_cut, dtype: int64

In [14]:
cols = ['cut','top10']

In [15]:
def make_streak_columns(cols):
    '''
    this function takes the columns that you want to count the streaks for and make them.
    
    input a list of columns that are in a dataframe that you want to find the streaks for.
    
    '''
    streak_df = pd.DataFrame()
    streak_dict = {}
    for i in cols:
        df = generate_streak_counter(golf_df,i)
        streak_dict[df.name] = df.values
        streak_df = pd.DataFrame.from_dict(streak_dict)
    return streak_df

In [16]:
streak_df = make_streak_columns(cols)

In [17]:
streak_df

,streak_counter_cut,streak_counter_top10
0,1,1
1,2,2
2,3,3
3,4,4
4,5,5
...,...,...
35540,5,22
35541,6,23
35542,1,24
35543,1,1


In [18]:
df_sorted

,tournament_id,player_id,date,golfer,cut,top10
0,2271,16,2014-10-12,Aaron Baddeley,1,0
1,2260,16,2014-10-19,Aaron Baddeley,1,0
2,2276,16,2014-10-26,Aaron Baddeley,1,0
3,2270,16,2014-11-16,Aaron Baddeley,1,0
4,2232,16,2015-02-01,Aaron Baddeley,1,0
...,...,...,...,...,...,...
35540,401025264,8889,2018-08-29,Zecheng Dou,0,0
35541,3747,8889,2018-11-14,Zecheng Dou,0,0
35542,401148241,8889,2019-11-03,Zecheng Dou,1,0
35543,401148241,8887,2019-11-03,Zheng-kai Bai,1,0


In [19]:
full_data = df_sorted.join(streak_df)

In [20]:
full_data.head(30)

,tournament_id,player_id,date,golfer,cut,top10,streak_counter_cut,streak_counter_top10
0,2271,16,2014-10-12,Aaron Baddeley,1,0,1,1
1,2260,16,2014-10-19,Aaron Baddeley,1,0,2,2
2,2276,16,2014-10-26,Aaron Baddeley,1,0,3,3
3,2270,16,2014-11-16,Aaron Baddeley,1,0,4,4
4,2232,16,2015-02-01,Aaron Baddeley,1,0,5,5
5,2234,16,2015-02-08,Aaron Baddeley,0,0,1,6
6,2233,16,2015-02-15,Aaron Baddeley,1,0,1,7
7,2236,16,2015-02-22,Aaron Baddeley,0,0,1,8
8,2261,16,2015-03-29,Aaron Baddeley,1,0,1,9
9,2244,16,2015-04-05,Aaron Baddeley,0,0,1,10


In [21]:
made_cut = full_data[full_data['cut']==1]

In [22]:
made_cut = made_cut.groupby(['golfer']).agg({'streak_counter_cut':['mean','max']})

In [23]:
made_cut = made_cut.reset_index()

In [24]:
made_cut

golfer streak_counter_cut    
                                  mean max
0    Aaron Baddeley           2.012821   7
1         Aaron Rai           1.923077   4
2        Aaron Wise           3.166667  14
3     Abraham Ancer           3.838710  15
4       Adam Hadwin           4.902256  17
..              ...                ...  ..
455    Zach Johnson           4.048000  14
456     Zack Sucher           1.480000   3
457  Zander Lombard           1.500000   2
458     Zecheng Dou           1.333333   3
459   Zheng-kai Bai           1.500000   2

[460 rows x 3 columns]

In [25]:
# SELECT golfer,MAX(streak_counter_cut) FROM streak_table WHERE cut = 1 GROUP BY golfer ORDER BY max DESC LIMIT 5;
#       golfer      | max
# ------------------+-----
#  Tommy Fleetwood  |  31
#  Dustin Johnson   |  31
#  Emiliano Grillo  |  27
#  Paul Casey       |  26
#  Hideki Matsuyama |  25
# (5 rows)
# #top 5 cuts made in a row

# SELECT golfer,MAX(streak_counter_cut) FROM streak_table WHERE cut = 0 GROUP BY golfer ORDER BY max DESC LIMIT 5;
#      golfer     | max
# ----------------+-----
#  Michael Kim    |  21
#  Matt Every     |  21
#  Brendon Todd   |  18
#  Nick Watney    |  17
#  Smylie Kaufman |  16
# (5 rows)
# #top 5 golfers that missed the cut in a row

# SELECT golfer,MAX(streak_counter_top10) FROM streak_table WHERE top10 = 1 GROUP BY golfer ORDER BY max DESC LIMIT 5;
#       golfer       | max
# -------------------+-----
#  Bryson DeChambeau |   7
#  Rory McIlroy      |   6
#  Dustin Johnson    |   6
#  Patrick Cantlay   |   6
#  Daniel Berger     |   5
# (5 rows)
# #top 5 golfers that made top 10 in a row

# SELECT golfer,MAX(streak_counter_top10) FROM streak_table WHERE top10 = 0 GROUP BY golfer ORDER BY max DESC LIMIT 5;
#     golfer     | max
# ---------------+-----
#  Peter Malnati | 127
#  Fabian Gomez  | 118
#  Billy Hurley  | 104
#  Brice Garnett |  92
#  Greg Chalmers |  90
# (5 rows)
# #top 5 golfers to miss the top 10 in a row



# #answers queried in SQL

In [26]:
#35 unique golfers that have had tournament top 10 streaks >=3 and 83 total times a streak 3>= has happened
#take the average strokes gained and see how the golfers performed compared to the average hot golfer the 
#game after the streak





In [27]:
#plot this data to visualize which golfer has made the highest amount of cuts and who has the highest average streaks
#460 golfers have made the cut at least 1 time

In [28]:
missed_cut = full_data[full_data['cut']==0]

In [29]:
missed_cut = missed_cut.groupby(['golfer']).agg({'streak_counter_cut':['mean','max']})

In [30]:
missed_cut = missed_cut.reset_index()

In [31]:
missed_cut

golfer streak_counter_cut    
                                   mean max
0     Aaron Baddeley           1.800000   5
1          Aaron Rai           1.375000   3
2         Aaron Wise           1.550000   5
3      Abel Gallegos           1.500000   2
4      Abraham Ancer           1.266667   3
..               ...                ...  ..
481     Zach Johnson           1.200000   3
482  Zachary Bauchou           2.500000   4
483      Zack Sucher           2.111111   5
484   Zander Lombard           1.000000   1
485      Zecheng Dou           2.400000   6

[486 rows x 3 columns]

In [32]:
#486 golfers have missed the cut
#this means 26 golfers never made the cut, unless some players never missed the cut in a tournament 

In [33]:
pd.set_option('display.max_rows', None)

In [34]:
full_data[full_data['golfer']=='Tommy Fleetwood']
#5/13/2018 to 2/29/2020

,tournament_id,player_id,date,golfer,cut,top10,streak_counter_cut,streak_counter_top10
32726,2267,5539,2014-11-09,Tommy Fleetwood,1,0,1,1
32727,2266,5539,2015-03-08,Tommy Fleetwood,1,0,2,2
32728,2247,5539,2015-06-07,Tommy Fleetwood,0,0,1,3
32729,2249,5539,2015-06-21,Tommy Fleetwood,1,0,1,4
32730,2253,5539,2015-07-20,Tommy Fleetwood,0,0,1,5
32731,2255,5539,2015-08-16,Tommy Fleetwood,0,0,2,6
32732,2519,5539,2015-11-08,Tommy Fleetwood,1,0,1,7
32733,2505,5539,2016-07-10,Tommy Fleetwood,0,0,1,8
32734,3735,5539,2017-03-05,Tommy Fleetwood,1,1,1,1
32735,2697,5539,2017-03-19,Tommy Fleetwood,1,1,2,2


In [35]:
full_data[full_data['golfer']=='Dustin Johnson']
#rare for dustin johnson to miss the cut 3/8/2015 to 7/24/2016

,tournament_id,player_id,date,golfer,cut,top10,streak_counter_cut,streak_counter_top10
10797,2234,3448,2015-02-08,Dustin Johnson,0,0,1,1
10798,2233,3448,2015-02-15,Dustin Johnson,1,1,1,1
10799,2236,3448,2015-02-22,Dustin Johnson,1,1,2,2
10800,2237,3448,2015-03-02,Dustin Johnson,0,0,1,1
10801,2266,3448,2015-03-08,Dustin Johnson,1,0,1,2
10802,2261,3448,2015-03-29,Dustin Johnson,1,1,2,1
10803,2241,3448,2015-04-12,Dustin Johnson,1,0,3,1
10804,2240,3448,2015-05-10,Dustin Johnson,1,0,4,2
10805,2245,3448,2015-05-31,Dustin Johnson,1,1,5,1
10806,2247,3448,2015-06-07,Dustin Johnson,1,0,6,1


In [36]:
# full_data.to_csv('streak_data.csv')

In [37]:
df = pd.read_csv('/dsa/groups/casestudy2022su/team06/streak_data.csv')

In [38]:
df.head()

,Unnamed: 0,tournament_id,player_id,date,golfer,cut,top10,streak_counter_cut,streak_counter_top10
0,0,2271,16,2014-10-12,Aaron Baddeley,1,0,1,1
1,1,2260,16,2014-10-19,Aaron Baddeley,1,0,2,2
2,2,2276,16,2014-10-26,Aaron Baddeley,1,0,3,3
3,3,2270,16,2014-11-16,Aaron Baddeley,1,0,4,4
4,4,2232,16,2015-02-01,Aaron Baddeley,1,0,5,5


In [39]:
df = df.drop(['Unnamed: 0'], axis =1)

In [40]:
df['date'] =  pd.to_datetime(df['date'])

In [41]:
df.dtypes
#make sure date is a date datatype

tournament_id                    int64
player_id                        int64
date                    datetime64[ns]
golfer                          object
cut                              int64
top10                            int64
streak_counter_cut               int64
streak_counter_top10             int64
dtype: object

In [42]:
import getpass
import pandas.io.sql as psql
mypasswd = getpass.getpass()
username = 'sebmb'
host = 'pgsql.dsa.lan'
database = 'casestdysu22t06'

········


In [43]:
# import pandas.io.sql as psql
# tourn_dates = psql.read_sql("SELECT DISTINCT * FROM tournament_table;",engine)
# tourn_dates.head()

In [44]:
from sqlalchemy.engine.url import URL
from sqlalchemy import create_engine
from sqlalchemy import select

# SQLAlchemy Connection Parameters
postgres_db = {'drivername': 'postgres',
               'username': username,
               'password': mypasswd,
               'host': host,
               'database' :database}
engine = create_engine(URL(**postgres_db), echo=True)
del mypasswd

In [45]:
print('streak_table')
engine.execute("SELECT golfer,cut,streak_counter_cut FROM streak_table WHERE golfer = 'Aaron Baddeley LIMIT 5';").fetchall()

streak_table
2022-07-07 07:45:00,501 INFO sqlalchemy.engine.base.Engine select version()
2022-07-07 07:45:00,502 INFO sqlalchemy.engine.base.Engine {}
2022-07-07 07:45:00,505 INFO sqlalchemy.engine.base.Engine select current_schema()
2022-07-07 07:45:00,506 INFO sqlalchemy.engine.base.Engine {}
2022-07-07 07:45:00,508 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2022-07-07 07:45:00,509 INFO sqlalchemy.engine.base.Engine {}
2022-07-07 07:45:00,513 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2022-07-07 07:45:00,514 INFO sqlalchemy.engine.base.Engine {}
2022-07-07 07:45:00,515 INFO sqlalchemy.engine.base.Engine show standard_conforming_strings
2022-07-07 07:45:00,515 INFO sqlalchemy.engine.base.Engine {}
2022-07-07 07:45:00,517 INFO sqlalchemy.engine.base.Engine SELECT golfer,cut,streak_counter_cut FROM streak_table WHERE golfer = 'Aaron Baddeley LIMIT 5';
2022-07-07 07:45:00,517 INFO sqla

[]

In [ ]:
engine.execute("grant select on player_table to public;")
engine.execute("grant select on streak_table to public;")
engine.execute("grant select on tournament_table to public;")
engine.execute("grant select on tourney_dates to public;")
engine.execute("grant select on weather_locations to public;")
engine.execute("grant select on weather_table to public;")
#grant permission for my team to all my tables in the database

In [ ]:
# SELECT t.tournament_id, t.player_id, s.golfer, s.date, t.shots_gained_total, t.cut, t.top10, s.streak_counter_cut, 
# s.streak_counter_top10 FROM tournament_table t JOIN streak_table s USING(tournament_id, player_id) LIMIT 5;





In [ ]:
# SELECT * FROM tournament_table t JOIN tourney_dates USING(tournament_id)  JOIN player_table USING(player_id)
# JOIN streak_table s USING(tournament_id, player_id)
# JOIN weather_table w USING(date) JOIN weather_locations wl USING(station) LIMIT 5;

In [ ]:
# SELECT * FROM tournament_table t JOIN tourney_dates td ON(t.tournament_id=td.tournament_id)
# JOIN player_table p ON(t.player_id = p.player_id)
# JOIN pga_majors pm ON(td.date = pm.end_date)
# JOIN streak_table s ON (t.tournament_id = s.tournament_id AND t.player_id = s.player_id) 
# JOIN weather_table w ON(td.date = w.date) JOIN weather_locations wl ON(w.station = wl.station) LIMIT 5;
#this joined all the datasets together